# Знакомство со структорой tsad

## Создание собственной задачи (Task)

**Task** - это отдельная задача, имеющая минимальный независимый смысл. Например, приведение данных к единой частоте дискретизации. Из последовательного набора тасков набирает пайплайн, который уже решает полноценно задачу. 


Для начала нам необходимо импортировать необходимые элементы, в том числе класс ```Task``` от которого затем будем наследоваться и класс ```TaskResult``` в которой будем хранить результаты работы задачи. 

In [8]:
# сначала импортируем необходимые библиотеки 
try:
    import tsad 
except: 
    import sys
    sys.path.insert(1, '../')
    import tsad

# 
, TaskResult

import pandas as pd
import numpy as np

Если мы хотим, чтобы наша новая собственная задача работала в логике работе tsad, нам сделать следующее: 
* Определить класс таси (у нас он будет называться ```MyCustomTask```)  н, который будет отнаследован от класса Task структуры из ```tsad.base.task```
* Определить в нем 2 метода 
    * ```fit_predict```
    * ```predict```
* Необязательный шаг. Определить TaskResult (у нас он будет называться ```MyCustomTaskResult```) и занести в него всю желаемую для хранения информацию.

In [2]:
from tsad.base.datasets import load_exhauster_faults


In [3]:
dataset = load_exhauster_faults()
df = dataset.frame[:1000]
df.iloc[5,5] = np.nan

In [13]:
from tsad.base.task import Task

class DropnaTask(Task):
    """
    Кастомная задача, которая в данном 
    случае будет дропать строку, если 
    там есть хотя  бы 1 нан. 
    """
    def fit_predict(self,df):
        # реализация обучения задачи
        new_df = df.dropna()
        return new_df

    def predict(self,df):
        # реализация предсказания задачи
        new_df = df.dropna()
        return dnew_dff


In [14]:
task = DropnaTask()
new_df = task.fit_predict(df)
df.shape, new_df.shape 

((1000, 16), (999, 16))

In [15]:
from tsad.base.task import Task, TaskResult

class ResampleTaskResult(TaskResult):
    freq: pd.Timedelta
        
    def show():
        pass

class ResampleTask(Task):
    """
    Кастомная задача, которая в данном случае будет 
    ресемплировать датасет до самой частой частоты
    """
    def fit_predict(self,df):
        # реализация обучения задачи
        freq = df.index.to_series().diff().value_counts().index[0]
        
        results = ResampleTaskResult()
        results.freq = freq
        
        new_df = df.resample(results.freq).mean()
        return new_df,results

    def predict(self,df,results:ResampleTaskResult):
        # реализация предсказания задачи
        new_df = df.resample(results.freq).mean()
        return new_df,results


In [28]:
task = ResampleTask()
new_df, result = task.fit_predict(df)
print(result.freq)
new_df, result = task.predict(new_df,result)
new_df.head(2)

0 days 00:00:10


,Rotor Current 1,Rotor Current 2,Stator Current,Oil Pressure in System,Bearing Temperature on Support 1,Bearing Temperature on Support 2,Bearing Temperature on Support 3,Bearing Temperature on Support 4,Oil Temperature in System,Oil Temperature in Oil Block,Vibration on Support 1,Vibration on Support 2,Vibration on Support 3,Vibration on Support 3. Longitudinal.,Vibration on Support 4,Vibration on Support 4. Longitudinal.
DT,,,,,,,,,,,,,,,,
2000-01-16 22:46:10,396.407567,396.407567,266.045020,130.12965,45.09330,43.32750,43.327500,48.547017,36.253400,44.210400,1.744000,0.828400,0.730300,1.776700,3.7605,0.806600
2000-01-16 22:46:20,395.782633,395.782633,264.905425,130.19444,45.09831,43.32977,43.595048,48.900729,36.343951,44.037178,1.714479,0.819392,0.700174,1.698394,3.7823,0.800469


## Pipiline

In [21]:
from tsad.base.pipeline import Pipeline
from tsad.base.wrappers import SklearnWrapper
from sklearn.preprocessing import StandardScaler

StandardScalerTask = SklearnWrapper(StandardScaler)

In [30]:
set_of_preproc_tasks = [
    ResampleTask(),
    DropnaTask(),
    StandardScalerTask(),
]

preproc_pipeline = Pipeline(set_of_preproc_tasks, 
                        show=False)
preprocessed_df = preproc_pipeline.fit_predict(df)
preprocessed_df.head(2)

,Rotor Current 1,Rotor Current 2,Stator Current,Oil Pressure in System,Bearing Temperature on Support 1,Bearing Temperature on Support 2,Bearing Temperature on Support 3,Bearing Temperature on Support 4,Oil Temperature in System,Oil Temperature in Oil Block,Vibration on Support 1,Vibration on Support 2,Vibration on Support 3,Vibration on Support 3. Longitudinal.,Vibration on Support 4,Vibration on Support 4. Longitudinal.
DT,,,,,,,,,,,,,,,,
2000-01-16 22:46:10,3.030539,3.030601,-0.666630,-1.267433,-0.254604,0.740498,-1.032639,-0.165575,-0.023516,1.095630,1.326801,1.332755,0.434482,-0.214912,0.768363,0.273307
2000-01-16 22:46:20,2.726841,2.726903,-0.916973,-0.960973,-0.223757,0.753434,0.699269,0.563728,0.168865,0.580937,0.360400,0.910253,-0.248955,-0.750909,0.801370,0.143023


# Разрыв сложного пайплайна

In [31]:
from tsad.tasks.eda import HighLevelDatasetAnalysisTask, TimeDiscretizationTask
from tsad.tasks.eda import FindNaNTask, EquipmentDowntimeTask
from tsad.tasks.preprocess import ResampleProcessingTask 
from tsad.tasks.preprocess import SplitByNaNTask, PrepareSeqSamplesTask
from tsad.tasks.deep_learning_anomaly_detection import ResidualAnomalyDetectionTask
from tsad.tasks.deep_learning_forecasting import DeepLeaningTimeSeriesForecastingTask

from tsad.base.pipeline import Pipeline
from tsad.base.wrappers import SklearnWrapper

from sklearn.preprocessing import StandardScaler
StandardScalerTask = SklearnWrapper(StandardScaler)



/opt/anaconda3/envs/tsad/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
preprocessing_set = [HighLevelDatasetAnalysisTask(),
                     TimeDiscretizationTask(freq_tobe_approach='most_frequent'),
                     FindNaNTask(),EquipmentDowntimeTask(),
                     ResampleProcessingTask(),
                     StandardScalerTask(),
                     PrepareSeqSamplesTask(len_seq=10),
                     SplitByNaNTask(),
                    ]
preprocessing_pipeline = Pipeline(preprocessing_set)
preprocessing_df = preprocessing_pipeline.fit_predict(dataset.frame)

In [ ]:
ml_modeling_set = [
    ResidualAnomalyDetectionTask()
]

preprocess_pipeline = Pipeline(mlmodeling_set, results=preprocessing_pipeline.results)
preprocess_fit_df = preprocess_pipeline.fit_predict(preprocessing_df)

In [ ]:
!